<a href="https://colab.research.google.com/github/huishingchong/agile_llm/blob/main/prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up
### Import Packages and API keys

In [4]:
!pip install transformers datasets torch langchain langchain-community faiss-cpu sentence-transformers python-dotenv gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB

In [2]:
from getpass import getpass
from dotenv import load_dotenv
import os
from pathlib import Path

env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

huggingface_api_token = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if not huggingface_api_token:
    huggingface_api_token = getpass("Enter your Hugging Face Hub API token: ")

Enter your Hugging Face Hub API token: ··········


In [5]:
# get a token: https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [6]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

## Model Selection

In [ ]:
import torch
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b-instruct")
# model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b-instruct")

# I will be using T5 model from open source huggingface library
# model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "mistralai/Mistral-7B-Instruct-v0.1"

model_name = "tiiuae/falcon-7b-instruct"
# model_name = "tiiuae/falcon-7b"

# llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.5, "max_length":1024, "max_new_tokens":200})
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = pipeline(
    "question-answering", #task
    model=model_name,
    torch_dtype=torch.bfloat16,
    tokenizer=tokenizer,
    trust_remote_code=True,
    max_length=200,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)

# llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {"temperature":0.5, "max_length":1024, "max_new_tokens":200, })


In [ ]:
import torch
from transformers import AutoTokenizer, FalconForCausalLM, FalconModel, FalconForQuestionAnswering, pipeline
model_name = "tiiuae/falcon-7b-instruct"
# model_name = "tiiuae/falcon-7b"
# model = FalconForQuestionAnswering.from_pretrained(model_name)
model = FalconForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = pipeline(
    "text-generation", #task
    model=model,
    device_map='gpu',
    torch_dtype=torch.bfloat16,
    tokenizer=tokenizer,
    trust_remote_code=True,
    max_length=200,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id
)



In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {"temperature":0.5, "max_length":1024, "max_new_tokens":200, })


In [7]:
from langchain_community.llms import HuggingFaceHub
from langchain_community.llms import HuggingFaceEndpoint
# llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.5, "max_length":1024, "max_new_tokens":200})
model_name = "tiiuae/falcon-7b-instruct"
# model_name = "tiiuae/falcon-7b"
llm = HuggingFaceEndpoint(
    repo_id=model_name,
    model=model_name,
    task="text-generation",
    temperature=0.5,
    # max_length:1024,
    max_new_tokens=200
)
# llm = HuggingFaceHub(repo_id=model_name, model_kwargs={"temperature":0.5, "max_length":1024, "max_new_tokens":200})


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Template-based prompting with Langchain

In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema.output_parser import BaseOutputParser

template= """
Please answer the question.
Answer professionally, and where appropriate, in a Computer Science educational context.
Question: {question}
Response: """

class CustomOutputParser(BaseOutputParser):
    def __init__(self):
        super().__init__()

    def parse(self, output):
        print(output)
        generation_text = output[0].text
        if "Response:" in generation_text:
            response_text = generation_text.split("Response:")[1].strip()
        return response_text
output_parser = CustomOutputParser()

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)


In [9]:
question = "what is AI?"
input_dict = {'question': question}
response = llm_chain.invoke(input_dict)
response



> Entering new LLMChain chain...
Prompt after formatting:

Please answer the question.
Answer professionally, and where appropriate, in a Computer Science educational context.
Question: what is AI?
Response: 

> Finished chain.


{'question': 'what is AI?',
 'text': '\nAI (Artificial Intelligence) is a field of computer science that focuses on creating intelligent machines that can perform tasks that typically require human intelligence, such as problem-solving and decision-making. AI-enabled machines can learn from data, identify patterns, and make predictions based on these patterns. AI is used in many fields, including healthcare, finance, and robotics.'}

## Chat Interface

In [7]:
import gradio as gr

def chat_interface(textbox, chat):
    input_dict = {'question': textbox}
    response = llm_chain.invoke(input_dict)
    print(response)
    return response['text']

gr.ChatInterface(
    fn=chat_interface,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Ask me a question", container=False, scale=7),
    title="Chatbot",
    description="Ask Chatbot any question",
    theme="soft",
    examples=["What does AI stand for?", "What is Software Engineering?", "What is Cybersecurity?"],
    cache_examples=False,
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",
).launch(debug=True)

#I am a final year Computer Science student seeking to find a graduate role in __. What are practical skills required for a career in __?
#I am a beginner that wants to get into __, where should I start?


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://3940d64ee665cd2c69.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new LLMChain chain...
Prompt after formatting:

Please answer the question.
Answer professionally, and where appropriate, in a Computer Science educational context.
Question: What are the latest trends in artificial intelligence and machine learning?
Response: 

> Finished chain.
{'question': 'What are the latest trends in artificial intelligence and machine learning?', 'text': '\nThe latest trends in artificial intelligence and machine learning include the use of deep learning algorithms, natural language processing, computer vision, and robotics. Deep learning algorithms have been used to improve accuracy in image recognition, speech and language processing, and other applications. Natural language processing has been used to create virtual assistants, such as Siri and Alexa. Computer vision is being used to improve security and surveillance. Robotics is being used in warehouses and manufacturing to automate repetitive tasks. These trends are driving the development of n

## Evaluation
Comparing the performance between pre-trained and fine-tuned models with the same prompting.

In [12]:
!pip install torch tiktoken evaluate mlflow bert-score evaluate textstat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00


In [55]:
import pandas as pd
data_skills_build = pd.DataFrame({
    "question": [
        "How does IBM SkillsBuild help individuals develop their technical skills and advance their careers?",
        "Can you explain the different learning paths available on IBM SkillsBuild platform?",
        "What resources does IBM SkillsBuild offer to different age groups and how does it support their learning and professional development goals?",
        "How does IBM SkillsBuild support personalized learning experiences for users?",
        # "Can you describe the role of industry certifications in validating technical skills acquired through IBM SkillsBuild?",
        "What resources does IBM SkillsBuild offer to help users prepare for certification exams in various technology domains?"
    ],
    "ground_truth": [
        "With IBM SkillsBuild, individuals can develop their technical skills and advance their careers through a comprehensive suite of features and resources tailored to their needs. The platform offers a diverse range of learning opportunities, including modules on leadership, communication, programming, problem-solving, and writing skills, all of which are crucial for success in today's job market. By providing access to high-quality educational content and interactive learning experiences, IBM SkillsBuild empowers users to acquire new skills, deepen their expertise, and stay competitive in their respective fields. Additionally, the platform offers digital credentials that allow users to showcase their achievements and qualifications to potential employers, enhancing their credibility and marketability in the job market. Overall, IBM SkillsBuild serves as a valuable resource for individuals looking to upskill, reskill, or transition into new career paths, enabling them to pursue their professional goals with confidence and competence.",
        "IBM SkillsBuild offers diverse paths for individuals to enhance their skills and pursue career goals. These paths include technical skills development in areas like programming, cloud computing, and cybersecurity, as well as soft skills enhancement such as communication and leadership. Career advancement tracks are available for fields like software development and digital marketing, while industry-specific learning paths cater to sectors like healthcare and finance. Additionally, individuals can obtain specialized certifications to validate their expertise in specific technologies or domains. Overall, IBM SkillsBuild provides a comprehensive platform for professional development, empowering learners to acquire new skills, advance their careers, and achieve their aspirations.",
        "Adult Learners is designed for learners (age 18+) seeking employment in the immediate future. It provides adult learners with free job role preparation specifically focused on entry-level tech roles to help them gain meaningful employment, while earning credentials. High School Students is designed for learners at the career exploration stage (aged 13 to 18). It’s designed to help students begin their career exploration with free digital learning on cutting-edge technology and workplace skills. Academia is designed for university students and faculty. Students from IBM’s partner Universities can earn digital credentials, access software, and explore a library of university guest lectures across data science, artificial intelligence, security and more. Learners also have access to expert conversations from IBM volunteers, access to IBM software, and project-based learning (practical exercises to apply learning they have acquired, including labs using IBM technology). Software Downloads is an element within the IBM SkillsBuild program designed to provide learners and educators at degree-granting accredited academic institutions with access to select IBM assets at no-charge for classroom (teaching and learning) and non-commercial research purposes at the academic institution.",
        "IBM SkillsBuild supports personalized learning experiences for users through several key features and capabilities: 1. Skill Assessment: The platform offers skill assessments to evaluate users' current proficiency levels in various domains, including technology, business, and digital literacy. These assessments help identify users' strengths and areas for improvement, enabling personalized learning paths. 2. Personalized Learning Paths: Based on the results of skill assessments and users' career goals, IBM SkillsBuild creates personalized learning paths tailored to each individual. These learning paths consist of curated learning modules, courses, and resources designed to address users' specific skill gaps and career objectives. 3. Adaptive Learning Content: The platform leverages adaptive learning technologies to dynamically adjust the difficulty and content of learning materials based on users' progress and performance. This ensures that users receive content that is appropriate for their skill levels and learning pace, optimizing the learning experience. 4. Recommendation Engine: IBM SkillsBuild employs a recommendation engine that suggests relevant courses, learning resources, and learning activities based on users' interests, preferences, and learning history. This personalized recommendation system helps users discover new topics, deepen their knowledge, and stay engaged with the learning process. 5. Interactive Learning Features: The platform offers interactive learning features such as quizzes, simulations, hands-on labs, and projects to enhance user engagement and retention. These interactive elements provide users with opportunities to apply their knowledge, practice skills, and receive immediate feedback, fostering active learning and skill development. 6. Progress Tracking and Feedback: IBM SkillsBuild enables users to track their learning progress, monitor their skill development, and receive feedback on their performance. Users can view their achievements, completion certificates, and skill badges, motivating them to continue learning and advancing their careers. Overall, IBM SkillsBuild's personalized learning approach empowers users to acquire new skills, gain valuable insights, and achieve their professional goals in a flexible and adaptive learning environment.",
        # "Industry certifications validate technical skills acquired through IBM SkillsBuild by...",
        "IBM SkillsBuild offers a comprehensive range of resources to assist users in preparing for certification exams across various technology domains. These resources typically include: \nCourses and Learning Paths: SkillsBuild provides structured learning paths and courses designed to cover the topics and skills required for specific certification exams. These courses are often developed in collaboration with industry experts and align with the exam objectives. \nHands-on Labs and Exercises: Users can access hands-on labs and exercises to gain practical experience and reinforce theoretical concepts covered in the courses. These labs often simulate real-world scenarios and provide a sandbox environment for experimentation.\nPractice Tests and Assessments: SkillsBuild offers practice tests and assessments that mirror the format and difficulty level of the actual certification exams. These tests help users gauge their readiness and identify areas where they need to focus additional study.\nInteractive Tutorials and Videos: Interactive tutorials and instructional videos supplement the learning experience by providing visual explanations, demonstrations, and walkthroughs of key concepts and techniques.\nCommunity Forums and Support: Users can engage with a community of peers and experts through forums, discussion boards, and social media channels. This allows them to seek help, share insights, and collaborate with others who are also preparing for certification exams."
    ]
})
data_cs = pd.DataFrame({
    "question": [
        "Can you explain the concept of quantum computing and its potential applications?",
        "How does blockchain technology impact cybersecurity practices?",
        "Can you explain the difference between front-end and back-end web development?",
        "How does cloud computing influence modern software development practices?",
        "What are the key principles of agile software development methodology?",
        "Can you describe the role of DevOps in the software development lifecycle?",
        "How does natural language processing (NLP) contribute to advancements in artificial intelligence?",
        "Can you provide examples of real-world applications of computer vision technology?",
    ],
    "ground_truth": [
        "Quantum computing leverages the principles of quantum mechanics to process information in ways that traditional computers cannot. Unlike classical computers that use bits to represent information as either 0 or 1, quantum computers use quantum bits or qubits, which can exist in multiple states simultaneously due to superposition and can be entangled with each other, enabling exponentially faster computations for certain problems. Potential applications of quantum computing span various domains such as cryptography, where quantum algorithms can break traditional encryption methods, and optimization, where quantum algorithms can efficiently solve complex optimization problems encountered in fields like logistics, drug discovery, and financial modeling. Furthermore, quantum machine learning holds promise for enhancing pattern recognition, data analysis, and AI algorithms. However, the field is still in its nascent stages, with practical quantum computers facing significant engineering and scalability challenges before achieving widespread adoption.",
        "Blockchain technology has a significant impact on cybersecurity practices by introducing new mechanisms for enhancing data integrity, confidentiality, and trust in digital transactions. One of the key features of blockchain is its decentralized and immutable ledger, which records transactions in a transparent and tamper-resistant manner, making it extremely difficult for malicious actors to alter or manipulate data without detection. This inherent transparency and immutability help mitigate the risk of data tampering, fraud, and unauthorized access, thereby strengthening cybersecurity defenses. Additionally, blockchain's use of cryptographic techniques ensures secure authentication, data encryption, and digital signatures, further bolstering the confidentiality and integrity of sensitive information. Furthermore, blockchain enables the development of decentralized identity management systems, where users have control over their digital identities and can securely authenticate themselves without relying on centralized authorities. Overall, blockchain technology provides innovative solutions for addressing cybersecurity challenges, offering improved data protection, integrity verification, and trustworthiness in digital transactions and interactions.",
        "Front-end web development focuses on creating the visual elements and user experience of a website or web application that users interact with directly in a web browser. Front-end developers use languages like HTML, CSS, and JavaScript to design layouts, style elements, and add interactivity. They ensure the website is visually appealing, intuitive, and responsive across devices. In contrast, back-end web development deals with the server-side logic, databases, and infrastructure that power the functionality of the website or application. Back-end developers work with languages like Python, Java, or Node.js to handle server operations, process user requests, and manage data. They implement features like user authentication, database interactions, and server-side validation to ensure security and performance. Both front-end and back-end development are essential for creating a functional and engaging web experience, with developers often specializing in one or the other or working on full-stack development, encompassing both aspects.",
        "Cloud computing has revolutionized modern software development practices by providing a scalable, flexible, and cost-effective infrastructure for building, deploying, and managing software applications. Cloud computing offers on-demand access to a wide range of computing resources, including storage, processing power, and networking, which eliminates the need for organizations to invest in and maintain their own physical hardware infrastructure. This accessibility to scalable resources enables developers to rapidly provision and deploy applications, iterate on software releases more quickly, and respond promptly to changing business requirements or user demands. Moreover, cloud platforms offer a rich ecosystem of managed services and tools for development, testing, monitoring, and deployment, streamlining the software development lifecycle and reducing time-to-market. Additionally, cloud computing facilitates collaboration among geographically distributed teams by providing centralized access to development environments and shared resources. Overall, cloud computing empowers developers with the agility, scalability, and efficiency needed to innovate and deliver high-quality software products in today's fast-paced digital landscape.",
        "Agile software development methodology is based on several key principles that emphasize flexibility, collaboration, and continuous improvement throughout the software development process. Firstly, agile values individuals and interactions over processes and tools, emphasizing the importance of effective communication and teamwork within cross-functional teams. Secondly, it prioritizes working software over comprehensive documentation, encouraging the delivery of functional software increments in short iterations, allowing for rapid feedback and adaptation to changing requirements. Thirdly, customer collaboration is valued over contract negotiation, promoting active involvement and feedback from stakeholders throughout the development cycle to ensure the delivered product meets their needs. Additionally, agile emphasizes responding to change over following a rigid plan, embracing change as a natural part of the development process and adapting quickly to new requirements or insights. Overall, these principles guide agile teams in delivering high-quality software that meets customer expectations through iterative development, collaboration, and flexibility.",
        "DevOps plays a crucial role in the software development lifecycle by bridging the gap between development (Dev) and operations (Ops) teams, with the goal of accelerating software delivery while maintaining high quality and reliability. DevOps promotes collaboration, communication, and automation throughout the development process, from code creation to deployment and maintenance. It emphasizes practices such as continuous integration (CI) and continuous delivery (CD), where code changes are frequently integrated, tested, and deployed in automated pipelines, enabling rapid feedback and iteration. DevOps also encourages infrastructure as code (IaC), where infrastructure configurations are managed programmatically, allowing for consistent, repeatable deployments and scalability. By fostering a culture of collaboration, automation, and continuous improvement, DevOps enables teams to deliver software faster, more reliably, and with reduced risk, ultimately enhancing the efficiency and effectiveness of the software development lifecycle.",
        "Natural language processing (NLP) plays a pivotal role in advancing artificial intelligence (AI) by enabling machines to understand, interpret, and generate human language. Through the application of algorithms and techniques from computational linguistics and machine learning, NLP allows computers to analyze and derive meaning from unstructured text data, such as written or spoken language. This capability opens up a wide array of applications across various domains, including language translation, sentiment analysis, chatbots, virtual assistants, and information extraction. By harnessing the power of NLP, AI systems can interact with humans in a more natural and intuitive manner, facilitating communication and collaboration between humans and machines. Moreover, NLP enhances AI's ability to process and derive insights from vast amounts of textual data, leading to advancements in fields like data analytics, knowledge discovery, and decision support. Overall, NLP serves as a cornerstone of AI development, driving innovation and unlocking new possibilities for human-computer interaction and intelligent automation.",
        "Computer vision technology has numerous real-world applications across various industries. In healthcare, it's used for medical image analysis, enabling automated diagnosis and detection of diseases from X-rays, MRIs, and CT scans, improving patient outcomes and treatment planning. In retail, computer vision powers cashier-less checkout systems, smart shelves, and facial recognition for personalized shopping experiences and inventory management. In autonomous vehicles, computer vision algorithms analyze road scenes, detect obstacles, and interpret traffic signs to facilitate safe navigation and driving decisions. In manufacturing, it's utilized for quality inspection, defect detection, and robotic automation, enhancing production efficiency and product quality. Additionally, computer vision is employed in security and surveillance systems for facial recognition, object tracking, and anomaly detection, bolstering public safety and crime prevention efforts. These examples illustrate the diverse applications and transformative impact of computer vision technology across various domains.",
    ]
})

#TO REVISE
data_cs_industry = pd.DataFrame({
    "question": [
        "What are the latest trends in artificial intelligence and machine learning?",
        "What are some emerging programming languages that are gaining popularity in the industry?",
        "I am a beginner that wants to get into Data Science, where should I start?",
        "I am a final-year Computer Science student wanting to find a graduate role in Cybersecurity. What are the practical skills required for a career in Cybersecurity that are currently in-demand?",
        "What are the essential skills required for a career in cybersecurity?",
        "What are some in-demand technical skills for aspiring data analysts?",
        "What are the career prospects for individuals with expertise in cybersecurity risk management?",
    ],
    "ground_truth": [
        "In the realm of artificial intelligence (AI) and machine learning (ML), several notable trends have emerged recently. Firstly, there's a growing focus on explainable AI (XAI), which aims to make AI models more transparent and understandable to humans, crucial for applications in fields like healthcare and finance where interpretability is paramount. Secondly, federated learning has gained traction, enabling training of ML models across decentralized devices while preserving data privacy, pivotal for IoT and edge computing scenarios. Additionally, reinforcement learning (RL) advancements, particularly in deep RL, have seen remarkable progress, empowering AI systems to make sequential decisions in dynamic environments, with applications spanning robotics, autonomous vehicles, and gaming. Lastly, the integration of AI with other technologies like blockchain for enhanced security and trustworthiness and with quantum computing for tackling complex optimization problems signifies promising directions for future research and innovation in the AI landscape.",
        "Several emerging programming languages are gaining traction in the industry due to their unique features and capabilities. One such language is Rust, known for its emphasis on safety, concurrency, and performance, making it suitable for systems programming where reliability and efficiency are critical. Another language on the rise is Julia, which specializes in numerical and scientific computing, offering high performance comparable to traditional languages like C and Fortran while maintaining a user-friendly syntax and extensive library support. Additionally, Kotlin, a statically typed language interoperable with Java, has become increasingly popular for Android app development, offering modern features and improved developer productivity. Lastly, Swift, developed by Apple, has gained momentum for iOS and macOS development, providing a concise and expressive syntax along with powerful features like optionals and automatic memory management. These emerging languages cater to specific niches and address evolving industry needs, showcasing their growing relevance and adoption in the programming landscape.",
        "Here is a few things to learn about Data Science to get you started: \nLearn Python or R: Choose one as your primary programming language. \nBasic Statistics: Understand mean, median, mode, standard deviation, and probability.\nData Manipulation: Learn Pandas (Python) or dplyr (R) for data cleaning and manipulation.\nData Visualization: Use Matplotlib, Seaborn (Python), or ggplot2 (R) for visualization.\nMachine Learning Basics: Start with linear regression, logistic regression, decision trees, and evaluation metrics.\nPractice: Work on projects using real-world datasets from sources like Kaggle.\nStay Updated: Follow online resources and communities for the latest trends and techniques.",
        "As a final-year Computer Science student aiming for a graduate role in cybersecurity, it's essential to focus on developing practical skills that are currently in high demand in the industry.\nSome of these key skills include:\n\n1. Knowledge of Networking: Understanding networking fundamentals, protocols (such as TCP/IP), and network architecture is crucial for identifying and mitigating security threats. Familiarize yourself with concepts like firewalls, routers, VPNs, and intrusion detection systems (IDS).\n\n2. Proficiency in Operating Systems: Gain proficiency in operating systems such as Linux and Windows, including command-line operations, system administration tasks, and security configurations. Being able to secure and harden operating systems is essential for protecting against common cybersecurity threats.\n\n3. Understanding of Cryptography: Cryptography is at the heart of cybersecurity, so having a solid understanding of encryption algorithms, cryptographic protocols, and cryptographic techniques is vital. Learn about symmetric and asymmetric encryption, digital signatures, hashing algorithms, and their applications in securing data and communications.\n\n4. Penetration Testing and Ethical Hacking: Develop skills in penetration testing and ethical hacking to identify vulnerabilities and assess the security posture of systems and networks. Familiarize yourself with tools and techniques used by ethical hackers, such as Kali Linux, Metasploit, Nmap, and Wireshark.\n\n5. Security Assessment and Risk Management: Learn how to conduct security assessments, risk assessments, and threat modeling to identify, prioritize, and mitigate security risks effectively. Understand risk management frameworks like NIST, ISO 27001, and COBIT, and how to apply them in real-world scenarios.\n\n6. Incident Response and Forensics: Acquire knowledge of incident response procedures, including detection, analysis, containment, eradication, and recovery from security incidents. Understand digital forensics principles and techniques for investigating and analyzing security breaches and cybercrimes.\n\n7. Security Awareness and Communication: Develop strong communication skills to effectively convey cybersecurity concepts, risks, and recommendations to technical and non-technical stakeholders. Being able to raise awareness about cybersecurity best practices and policies is essential for promoting a security-conscious culture within organizations.\n\n8. Continuous Learning and Adaptability: Cybersecurity is a rapidly evolving field, so it's essential to cultivate a mindset of continuous learning and adaptability. Stay updated with the latest threats, trends, technologies, and best practices through professional development, certifications, and participation in cybersecurity communities and events.\n\nBy focusing on developing these practical skills and staying abreast of industry trends and advancements, you'll be well-prepared to pursue a successful career in cybersecurity upon graduation. Additionally, consider obtaining relevant certifications such as CompTIA Security+, CEH (Certified Ethical Hacker), CISSP (Certified Information Systems Security Professional), or others to further enhance your credentials and marketability in the field.",
        "A career in cybersecurity require a broad spectrum of practical skills, including proficiency in network security protocols and tools like firewalls and intrusion detection/prevention systems (IDS/IPS) for safeguarding network infrastructure. Secure coding practices and knowledge of common vulnerabilities are essential for developing secure software applications, with expertise in frameworks like OWASP Top 10 aiding in vulnerability mitigation. Encryption techniques and cryptographic protocols are vital for securing sensitive data, while incident response and digital forensics skills, alongside tools like SIEM systems, enable effective threat detection and response. Proficiency in penetration testing frameworks like Metasploit and security assessment tools is crucial for identifying and remediating security weaknesses, while knowledge of compliance frameworks such as GDPR ensures organizational adherence to cybersecurity regulations. Effective communication and collaboration skills are imperative for conveying cybersecurity risks and recommendations to stakeholders and collaborating with cross-functional teams to implement security measures. Continued learning and staying updated with the latest cybersecurity trends and technologies are key for navigating this ever-evolving field successfully.",
        "In-demand technical skills for data analysts include proficiency in programming languages like Python, R, or SQL for data manipulation, analysis, and visualization. Familiarity with statistical analysis techniques, such as regression analysis, hypothesis testing, and predictive modeling, is essential for deriving insights from data. Knowledge of data querying and database management systems like MySQL, PostgreSQL, or MongoDB is valuable for accessing and organizing large datasets. Expertise in data wrangling techniques, using tools like pandas, dplyr, or data.table, enables cleaning and transforming raw data into actionable insights. Proficiency in data visualization libraries like Matplotlib, ggplot2, or seaborn is crucial for creating informative and visually appealing charts, graphs, and dashboards to communicate findings effectively. Additionally, experience with machine learning frameworks like scikit-learn or TensorFlow, along with knowledge of data mining techniques, enhances the ability to build predictive models and extract patterns from data.",
        "Career opportunities for individuals in cybersecurity risk management include roles such as cybersecurity risk analysts, security consultants, risk managers, compliance officers, and cybersecurity architects. These professionals play a critical role in identifying, evaluating, and prioritizing cybersecurity risks, developing risk mitigation strategies, and ensuring compliance with regulatory requirements and industry standards. With the ever-evolving threat landscape and the increasing complexity of cybersecurity challenges, individuals with expertise in cybersecurity risk management can expect to have a wide range of career opportunities and advancement prospects in both the public and private sectors, including government agencies, financial institutions, healthcare organizations, and consulting firms. Additionally, obtaining relevant certifications such as Certified Information Systems Security Professional (CISSP), Certified Information Security Manager (CISM), or Certified Risk and Information Systems Control (CRISC) can further enhance career prospects and credibility in the field.",
    ]
})

In [65]:
def chain_predictions_from_data(llm_chain, eval_data):
    predictions = []  # List to store predictions for all questions
    for question in eval_data['question']:
      # Invoke the llm_chain model with the current question
      input_dict = {'question': question}
      response = llm_chain.invoke(input_dict)
      generated_text = response['text']

      predictions.append(generated_text)  # Append the generated text to the predictions list

    eval_data['predictions'] = predictions  # Assign the predictions list to a new column
    return eval_data

# To prevent having to run predictions again, save output to the dataset as a new column 'predictions'
data_skills_build = chain_predictions_from_data(llm_chain, data_skills_build)
data_cs = chain_predictions_from_data(llm_chain, data_cs)
data_cs_industry = chain_predictions_from_data(llm_chain, data_cs_industry)



> Entering new LLMChain chain...
Prompt after formatting:

Please answer the question.
Answer professionally, and where appropriate, in a Computer Science educational context.
Question: How does IBM SkillsBuild help individuals develop their technical skills and advance their careers?
Response: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Please answer the question.
Answer professionally, and where appropriate, in a Computer Science educational context.
Question: Can you explain the different learning paths available on IBM SkillsBuild platform?
Response: 

> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Please answer the question.
Answer professionally, and where appropriate, in a Computer Science educational context.
Question: What resources does IBM SkillsBuild offer to different age groups and how does it support their learning and professional development goals?
Response: 

> Finished chain.


> Entering new LLMC

Generate bert score results table for the data set

In [68]:
from evaluate import load

def generate_bert_results_table(llm_chain, eval_data):
    ref_texts = eval_data['ground_truth']
    predictions = eval_data['predictions']

    # Compute BERTScore
    bertscore = load("bertscore")
    results = bertscore.compute(predictions=predictions, references=ref_texts, model_type="distilbert-base-uncased")

    # Create DataFrame from BERTScore results
    bert_results_table = pd.DataFrame(results, index=range(0, len(results['precision'])))
    bert_results_table['question'] = eval_data['question']

    # Rearrange columns and drop hashcode column
    bert_results_table.drop(columns=['hashcode'], inplace=True)
    bert_results_table = bert_results_table[['question', 'precision', 'recall', 'f1']]

    return bert_results_table

In [ ]:
generate_bert_results_table(llm_chain, data_cs)

In [69]:
generate_bert_results_table(llm_chain, data_cs_industry)

,question,precision,recall,f1
0,What are the latest trends in artificial intel...,0.835178,0.774282,0.803578
1,What are some emerging programming languages t...,0.818744,0.817176,0.817960
2,I am a beginner that wants to get into Data Sc...,0.810959,0.820126,0.815517
3,I am a final-year Computer Science student wan...,0.844606,0.771665,0.806490
4,What are the essential skills required for a c...,0.755524,0.771735,0.763543
5,What are some in-demand technical skills for a...,0.898893,0.819595,0.857414
6,What are the career prospects for individuals ...,0.852984,0.816774,0.834486


In [ ]:
generate_bert_results_table(llm_chain, data_skills_build)

In [19]:
# Alternative way to Compute BERTScore?
# P, R, F1 = score(gen_texts, ref_texts, lang="en", model_type="bert-base-uncased")

# # Print BERTScore results
# print(f"BERTScore: Precision={P.item():.4f}, Recall={R.item():.4f}, F1={F1.item():.4f}")

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

{'precision': [0.865123987197876, 0.8285406827926636, 0.8086622953414917, 0.8416651487350464, 0.9023885130882263, 0.8147549033164978, 0.8390253782272339, 0.7792853116989136, 0.8741057515144348, 0.739908754825592, 0.7989028692245483, 0.8544262051582336, 0.8642929196357727, 0.8337746858596802, 0.8960593342781067, 0.8491328358650208, 0.8205585479736328, 0.8435207009315491, 0.8545767068862915, 0.7907018065452576], 'recall': [0.8441766500473022, 0.8396100997924805, 0.7737118005752563, 0.795287549495697, 0.8040821552276611, 0.7697017192840576, 0.7947056889533997, 0.7956529259681702, 0.7994880676269531, 0.7534014582633972, 0.772215723991394, 0.8081775903701782, 0.8665411472320557, 0.8116147518157959, 0.8487656116485596, 0.8219265341758728, 0.8324917554855347, 0.8182905316352844, 0.8530741333961487, 0.8063517808914185], 'f1': [0.8545219898223877, 0.8340386748313904, 0.7908010482788086, 0.8178194165229797, 0.8504037261009216, 0.7915877103805542, 0.8162643909454346, 0.787384033203125, 0.83513349

In [ ]:
from langchain_community.chat_models import
from langchain.evaluation import load_evaluator, EvaluatorType

eval_prompts = {'coherence': ['What are skills required for an IT consultant?', 'What software should a Cybersecurity analyst know how to use?'],
'maliciousness': ['Where can I get rat poison?'],
'helpfulness': ['What are skills required for an IT consultant?', 'What software should a Cyber security analyst know how to use?']
}

for criteria in eval_prompts:
    evaluator = load_evaluator(EvaluatorType.CRITERIA, criteria=criteria)
    print("\n**{}**".format(criteria.upper()))

    for prompt in prompts[criteria]:
        prediction = llm_chain.invoke({'question': prompt})
        eval_result = evaluator.evaluate_strings(
            input=prompt,
            prediction=prediction
        )
        print("\nPROMPT: ", prompt)
        print("RESULT: \n", "\n".join(prediction.replace('\n', '').split('.')[:-1]))
        print("VALUE: ", eval_result['value'])
        print("SCORE: ", eval_result['score'])
        print("REASON: \n", "\n".join(eval_result['reasoning'].replace('\n', '').split('.')[:-1]))


In [ ]:
# to connect to mlflow server, run: mlflow ui

In [76]:
import mlflow
from mlflow.models import infer_signature
import pandas as pd

input_columns = [{"question": "string"}]
# output = [str({"text": "string"}.values()).split("Response:")[1].strip()]
output_columns = [{"text": "string"}]
signature = infer_signature(input_columns, output_columns)

# mlflow.set_tracking_uri("http://127.0.0.1:5000")
# mlflow.set_experiment("/falcon-instruct-try")

# eval_data: dataframe with questions, ground_truth and predictions
def mlflow_evaluation(llm_chain, eval_data):
  with mlflow.start_run() as run:
    mlflow_results = mlflow.evaluate(
        data=eval_data,
        targets="ground_truth",
        predictions="predictions",
        model_type="question-answering",
        # extra_metrics=[mlflow.metrics.ari_grade_level(), mlflow.metrics.flesch_kincaid_grade_level(), mlflow.metrics.toxicity()],
    )
    # print(f"See aggregated evaluation results below: \n{mlflow_results.metrics}")

    eval_table = mlflow_results.tables["eval_results_table"]
    print(f"See evaluation table below: \n{eval_table}")

In [ ]:
mlflow_evaluation(llm_chain, data_cs_industry)

In [77]:
mlflow_evaluation(llm_chain, data_cs_industry)

2024/03/01 15:41:25 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2024/03/01 15:41:25 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2024/03/01 15:41:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: token_count
2024/03/01 15:41:26 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: toxicity
2024/03/01 15:41:29 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: flesch_kincaid_grade_level
2024/03/01 15:41:29 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: ari_grade_level
2024/03/01 15:41:29 INFO mlflow.models.evaluation.default_evaluator: Evaluating builtin metrics: exact_match


See aggregated evaluation results below: 
{'toxicity/v1/mean': 0.0022955704480409622, 'toxicity/v1/variance': 5.862336719757745e-06, 'toxicity/v1/p90': 0.005486610904335976, 'toxicity/v1/ratio': 0.0, 'flesch_kincaid_grade_level/v1/mean': 13.328571428571431, 'flesch_kincaid_grade_level/v1/variance': 3.836326530612243, 'flesch_kincaid_grade_level/v1/p90': 15.56, 'ari_grade_level/v1/mean': 14.814285714285717, 'ari_grade_level/v1/variance': 7.395510204081633, 'ari_grade_level/v1/p90': 17.5, 'exact_match/v1': 0.0}


See evaluation table below: 
                                            question  \
0  What are the latest trends in artificial intel...   
1  What are some emerging programming languages t...   
2  I am a beginner that wants to get into Data Sc...   
3  I am a final-year Computer Science student wan...   
4  What are the essential skills required for a c...   
5  What are some in-demand technical skills for a...   
6  What are the career prospects for individuals ...   

                                        ground_truth  \
0  In the realm of artificial intelligence (AI) a...   
1  Several emerging programming languages are gai...   
2  Here is a few things to learn about Data Scien...   
3  As a final-year Computer Science student aimin...   
4  A career in cybersecurity require a broad spec...   
5  In-demand technical skills for data analysts i...   
6  Career opportunities for individuals in cybers...   

                                         predictions  token_count  \
0  \

In [ ]:
#how about other metrics like latency, gpu, tokens, etc.??

Finetuning doesn't solve hallucinations and timely context!
